from google.colab import drive
drive.mount('/content/drive')

In [1]:
import pandas as pd
import numpy as np
import string
import os
import sys
import shutil
import openai
#os.chdir('/content/drive/MyDrive/Sanjiv/IMP-OIC-Windowing')
from utils.extractframes import FrameExtractor
from . import graphene
directory_path = 'STAR_eval/Charades'
from STAR_eval.gpt_ask import run_gpt
main_ds = pd.read_json('STAR_eval/STAR_val.json')
main = main_ds.loc[:, ['question_id','question','video_id','start','end','answer', 'choices']]


/Users/sanjivjha/Code/IMP-OIC-Windowing/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ModuleNotFoundError: No module named 'utils'

In [9]:
CLASSES = ['airplane', 'animal', 'arm', 'bag', 'banana', 'basket', 'beach', 'bear', 'bed', 'bench', 'bike',
                'bird', 'board', 'boat', 'book', 'boot', 'bottle', 'bowl', 'box', 'boy', 'branch', 'building',
                'bus', 'cabinet', 'cap', 'car', 'cat', 'chair', 'child', 'clock', 'coat', 'counter', 'cow', 'cup',
                'curtain', 'desk', 'dog', 'door', 'drawer', 'ear', 'elephant', 'engine', 'eye', 'face', 'fence',
                'finger', 'flag', 'flower', 'food', 'fork', 'fruit', 'giraffe', 'girl', 'glass', 'glove', 'guy',
                'hair', 'hand', 'handle', 'hat', 'head', 'helmet', 'hill', 'horse', 'house', 'jacket', 'jean',
                'kid', 'kite', 'lady', 'lamp', 'laptop', 'leaf', 'leg', 'letter', 'light', 'logo', 'man', 'men',
                'motorcycle', 'mountain', 'mouth', 'neck', 'nose', 'number', 'orange', 'pant', 'paper', 'paw',
                'people', 'person', 'phone', 'pillow', 'pizza', 'plane', 'plant', 'plate', 'player', 'pole', 'post',
                'pot', 'racket', 'railing', 'rock', 'roof', 'room', 'screen', 'seat', 'sheep', 'shelf', 'shirt',
                'shoe', 'short', 'sidewalk', 'sign', 'sink', 'skateboard', 'ski', 'skier', 'sneaker', 'snow',
                'sock', 'stand', 'street', 'surfboard', 'table', 'tail', 'tie', 'tile', 'tire', 'toilet', 'towel',
                'tower', 'track', 'train', 'tree', 'truck', 'trunk', 'umbrella', 'vase', 'vegetable', 'vehicle',
                'wave', 'wheel', 'window', 'windshield', 'wing', 'wire', 'woman', 'zebra']

REL_CLASSES = ['above', 'across', 'against', 'along', 'and', 'at', 'attached to', 'behind',
                'belonging to', 'between', 'carrying', 'covered in', 'covering', 'eating', 'flying in', 'for',
                'from', 'growing on', 'hanging from', 'has', 'holding', 'in', 'in front of', 'laying on',
                'looking at', 'lying on', 'made of', 'mounted on', 'near', 'of', 'on', 'on back of', 'over',
                'painted on', 'parked on', 'part of', 'playing', 'riding', 'says', 'sitting on', 'standing on',
                'to', 'under', 'using', 'walking in', 'walking on', 'watching', 'wearing', 'wears', 'with']

words = set(CLASSES+REL_CLASSES)

In [4]:
import string
index_list = []
for q in main.loc[:,'question_id'].values:
    questions = main.query("question_id=='"+q+"'")["question"]
    answer = main.query("question_id=='"+q+"'")["answer"].values[0].lower().strip('the').translate(str.maketrans('', '', string.punctuation))
    count = 0
    question = ''
    for que in questions:
        question = que.lower().translate(str.maketrans('', '', string.punctuation))

    for i in list(answer.strip().split(' ')) + list(question.strip().split(' ')):
      if i not in  ['a', 'is','by', 'was', 'which', 'man', 'did', 'do', 'woman', 'boy', 'girl', 'person','people','scene','frame','in','to','the', 'of', 'on','with', 'from', 'at', 'and']:
          if i in words:
            index_list.append(main[main['question_id']==q].index[0])

print(index_list)
#QA.drop(index_list, inplace = True)
main = main[main.index.isin(index_list)]
main.info()


[3, 6, 8, 9, 13, 17, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 35, 36, 37, 41, 42, 43, 44, 45, 49, 50, 51, 52, 53, 54, 56, 57, 58, 60, 61, 64, 65, 66, 75, 76, 80, 81, 82, 85, 86, 90, 93, 94, 96, 97, 98, 99, 102, 104, 106, 107, 108, 109, 110, 111, 113, 114, 115, 116, 121, 128, 129, 131, 140, 141, 143, 144, 145, 147, 148, 149, 150, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 175, 176, 177, 178, 179, 180, 182, 184, 185, 187, 189, 190, 192, 195, 199, 200, 201, 202, 203, 204, 205, 206, 209, 210, 212, 217, 219, 220, 221, 222, 223, 229, 232, 236, 238, 239, 240, 243, 244, 245, 246, 247, 248, 254, 255, 256, 257, 258, 259, 261, 265, 266, 267, 269, 271, 272, 274, 277, 278, 290, 291, 292, 296, 297, 298, 303, 307, 308, 311, 312, 313, 318, 319, 321, 323, 326, 327, 328, 329, 330, 339, 345, 349, 350, 351, 352, 361, 363, 364, 367, 372, 374, 375, 376, 377, 379, 380, 382, 383, 386, 387, 390, 391, 394, 395, 400, 401, 402, 403, 405, 406, 407, 408, 41

In [5]:
%pip install ffmpeg-python
import ffmpeg

def segment_video(input_path, output_path, start, end):
    input_file = ffmpeg.input(directory_path+'/'+input_path+'.mp4')
    duration = int(end-start)
    if duration < 10:
      output_file = ffmpeg.output(input_file.trim(start=start, duration = duration).filter('setpts','PTS-STARTPTS'), directory_path+'/trim/'+output_path+'.mp4')
      ffmpeg.run(output_file)
      return True
    else:
      return False

In [6]:
def run_oic(question_id, video_id, question, choices, start,end):
    TEMP_DIR = "temp"
    CAM_PATH = "cam"
    OUT_DIR = "out"

    for video in os.scandir(directory_path):
        if video.is_file():
            if os.path.splitext(video.name)[0] == video_id:
                print('processing {}'.format(video_id))
                print(video_id, question, choices, start, end)
                video_p = segment_video(os.path.splitext(video.name)[0],question_id+os.path.splitext(video.name)[0],start,end)
                if video_p:
                  ex = FrameExtractor(directory_path+'/trim/'+question_id+video.name, 10, 5)
                  ex.main()
                  g = graphene.Graphene(0.3, 0.6)

                  if not os.path.isdir(OUT_DIR):
                      os.mkdir(OUT_DIR)
                  text = question_id+os.path.splitext(video.name)[0]+'graph2text.txt'
                  img_path = directory_path+'/trim/'+question_id+os.path.splitext(video.name)[0]+'-opencv'
                  #visual = question_id+os.path.splitext(video.name)[0]+'tg.png'
                  #g.run_online(text)


                  g.classify_images_window(img_path, 5)
                  graph_path = img_path + "/img/JSON"
                  g.generate_temporal_graph_frames(graph_path, img_path + "/img")

                  #g.generate_temporal_graph(graph_path)

                  #g.classify_images(img_path)
                  #graph_path = TEMP_DIR
                  #g.generate_temporal_graph_frames(graph_path, img_path)

                  #g.tg.to_plot(os.path.join(OUT_DIR, visual))

                  g.tg.to_text(os.path.join(OUT_DIR, text))
                  #if os.path.isdir(TEMP_DIR):
                  #    shutil.rmtree(TEMP_DIR)
                  if os.path.isfile(os.path.join(OUT_DIR,text)):
                      with open(os.path.join(OUT_DIR,text)) as f:
                          context = "".join(map(str,f.readlines()))
                          return str(context)
                  else:
                    continue

In [7]:
import shutil
shutil.rmtree('temp')

In [ ]:
import contextlib
import time
q_ids = main['question_id'].unique()
for q in q_ids:
    if 'Sequence' in q:
      choices = 'Options: '
      que = main.query("question_id == '"+q+"'")
      video_id = que['video_id'].values[0]
      question = que['question'].values[0]
      start = que['start'].values[0]
      end = que['end'].values[0]
      for choice in que['choices']:
          for c in choice:
              choices+=c['choice']
      if not os.path.isdir(directory_path+'/trim/'+q+video_id+'-opencv'):
        prompt = run_oic(q, video_id, question, choices, start, end)
        if prompt is not None:
          main.loc[main['question_id'] == q, 'OIC_context'] = prompt
          answer = run_gpt(prompt, question + choices)
          main.loc[main['question_id'] == q, 'OIC_answer'] = str(answer)
          #print('OIC answer: {}'.format(answer))
          main.to_csv('STAR_eval/question_answers.csv')


In [11]:
ds = pd.read_csv('STAR_eval/question_answers.csv')
ds.dropna(inplace=True)
ds.head()
import string
import time

q_ids = ds['question_id'].unique()

for q in q_ids:
  que = ds.query("question_id == '"+q+"'")
  answer = que['answer'].values[0].lower().strip('the ').translate(str.maketrans('', '', string.punctuation))
  OIC_answer = que['OIC_answer'].values[0]
  if answer not in OIC_answer:
    ds.loc[ds['question_id'] == q, 'Match'] = 'Wrong'
  else:
    ds.loc[ds['question_id'] == q, 'Match'] = 'Correct'

ds.to_csv('datatables/question_answers_checked'+str(time.time())+'.csv')
ds.head()

,Unnamed: 0,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,Match
4846,6810,Feasibility_T5_981,Which object is the person able to hold after ...,IO712,0.0,5.9,The book.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",In frame 0: man_02b0 wearing shirt_df93.man_02...,"Based on the description given, there is no me...",Correct


In [16]:
os.chdir('/content/drive/MyDrive/Sanjiv/IMP-OIC-Windowing/STAR_eval/datatables')
csv_files = [f for f in os.listdir() if f.endswith('.csv')]

In [17]:
dfs = []

for csv in csv_files:
    df = pd.read_csv(csv)
    dfs.append(df)

In [18]:
final_df = pd.concat(dfs, ignore_index=True)
final_df.drop_duplicates()
final_df.reindex()
final_df.head()

videos = final_df['question_id'].unique()
print(videos.size)

videos = main['video_id'].unique()
print(videos.size)


162
760


In [19]:
import string
index_list = []
for q in final_df.loc[:,'question_id'].values:
    questions = final_df.query("question_id=='"+q+"'")["question"]
    answer = final_df.query("question_id=='"+q+"'")["answer"].values[0].lower().strip('the').translate(str.maketrans('', '', string.punctuation))
    count = 0
    question = ''
    for que in questions:
        question = que.lower().translate(str.maketrans('', '', string.punctuation))

    for i in list(answer.strip().split(' ')) + list(question.strip().split(' ')):
      if i not in  ['a', 'is','by', 'was', 'which', 'man', 'did', 'do', 'woman', 'boy', 'girl', 'person','people','scene','frame','in','to','the', 'of', 'on','with', 'from', 'at', 'and']:
          if i in words:
            index_list.append(final_df[final_df['question_id']==q].index[0])

print(index_list)
#QA.drop(index_list, inplace = True)
final_df = final_df[final_df.index.isin(index_list)]
final_df.info()

[0, 1, 2, 6, 6, 9, 10, 13, 18, 21, 23, 24, 26, 28, 29, 30, 31, 33, 37, 40, 41, 41, 42, 43, 44, 45, 47, 47, 48, 48, 49, 50, 52, 54, 55, 56, 57, 58, 58, 59, 60, 63, 64, 65, 66, 67, 68, 70, 71, 73, 74, 75, 75, 76, 0, 1, 2, 56, 57, 58, 58, 59, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 99, 99, 100, 101, 29, 30, 31, 33, 37, 40, 114, 114, 115, 115, 116, 116, 117, 117, 118, 119, 120, 120, 122, 124, 76, 126, 126, 127, 127, 128, 128, 129, 129, 130, 130, 131, 131, 132, 132, 133, 133, 134, 134, 135, 135, 136, 136, 9, 10, 13, 116, 116, 117, 117, 148, 149, 150, 150, 151, 152, 152, 153, 154, 155, 155, 156, 157, 158, 159, 161, 162, 129, 129, 130, 130, 131, 131, 132, 132, 133, 133, 134, 134, 135, 135, 136, 136, 171, 171, 172, 173, 173, 174, 174, 29, 30, 31, 33, 37, 40, 187, 189, 189, 190, 191, 29, 30, 31, 33, 37, 40, 204, 205, 206, 207, 0, 1, 2, 214, 214, 215, 215, 215, 217, 217, 219, 220, 221, 222, 223, 97, 97, 226, 228, 229, 230, 233, 234, 235, 236, 29, 30, 31, 33, 56, 57, 58, 58, 59, 0, 1, 2, 189,

In [20]:
start = main['start']
end = main['end']

duration = end - start

print(duration.mean())

12.26367734282325


In [21]:
import string
import time

ds = final_df
q_ids = ds['question_id'].unique()
v_ids = ds['video_id'].unique()
ds = ds.drop(columns =['Match'])
for q in q_ids:
  que = ds.query("question_id == '"+q+"'")
  answer = que['answer'].values[0].lower().strip('the ').translate(str.maketrans('', '', string.punctuation))
  OIC_answer = que['OIC_answer'].values[0]
  filter_words = ['not', 'no', 'none']
  if 'not' in OIC_answer or 'no' in OIC_answer or 'none' in OIC_answer:
    ds.loc[ds['question_id'] == q, 'Match'] = 'Wrong'
  elif answer in OIC_answer:
      ds.loc[ds['question_id'] == q, 'Match'] = 'Correct'
  else:
    ds.loc[ds['question_id'] == q, 'Match'] = 'Wrong'

ds.head()
ds.name = 'all data'
print(v_ids.size)

87


In [22]:

for i in ds['question_id']:
  que = ds.query("question_id == '"+i+"'")
  questions = que["question"].values[0]
  answer = que["answer"].values[0].lower().strip('the').translate(str.maketrans('', '', string.punctuation))
  choices = dict()
  choice_string = ''
  options = main.query("question_id == '"+i+"'")["choices"].values[0]
  for choice in options:
        choices.update({choice['choice_id']:choice['choice'].lower().strip('the').translate(str.maketrans('', '', string.punctuation))})
        choice_string += ' ('+str(choice['choice_id']+1)+')'+str(choice['choice'].lower().strip('the').translate(str.maketrans('', '', string.punctuation)))

  formated_question = questions+' Guess the most likely answer among these four options: '+choice_string+' Respond only with a single number between 1 and 4. Do not produce any other output.'
  print('Question : {}'.format(formated_question))
  print('Expected Answer: {}'.format(answer))
  context = ds.query("question_id=='"+i+"'")["OIC_context"].values[0]
  print('Prompt for LLM: {}'.format(context))
  oic_answer = run_gpt(context, formated_question)
  print('OIC answer: choice- {}, vlaue- {}'.format(oic_answer, choices[int(oic_answer)-1]))

  if choices[int(oic_answer)-1] == answer:
    ds.loc[ds['question_id'] == i, 'new_Match'] = 'Correct'
    print('correct')
  else:
    print('wrong')
    ds.loc[ds['question_id'] == i, 'new_Match'] = 'Wrong'
  print('------------------------------------')





Question : What will the person do next? Guess the most likely answer among these four options:  (1)put down the cupglassbottle (2)wash the mirror (3)open the box (4)open the door Respond only with a single number between 1 and 4. Do not produce any other output.
Expected Answer: open the door
Prompt for LLM: In frame 0: man_9076 wearing jean_b44d.man_9076 wearing shirt_91f6.man_9076 has hair_820a.man_9076 wearing glove_54fe.shirt_91f6 on man_9076.window_7e2a behind man_9076.window_7e2a in room_133e.head_862a of man_9076.dog_7843 on shelf_046c.In frame 1: man_9076 wearing jean_b44d.man_9076 wearing shirt_91f6.man_9076 has hair_820a.man_9076 has hand_2d79.shirt_91f6 on man_9076.window_7e2a behind man_9076.head_862a of man_9076.dog_7843 on shelf_046c.In frame 2: man_9076 wearing jean_b44d.man_9076 wearing shirt_91f6.man_9076 has hair_820a.man_9076 wearing glove_54fe.man_9076 wearing shoe_ce7e.man_9076 has arm_60e6.jean_b44d on man_9076.shirt_91f6 on man_9076.window_7e2a behind man_9076.w

KeyboardInterrupt: ignored

In [23]:
interaction_ds = ds.loc[ds['question_id'].str.contains('Interaction')]
interaction_ds.name='Interaction'
interaction_ds.head()


,Unnamed: 0.1,Unnamed: 0,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,Match,new_Match
9,215,450,Interaction_T1_3788,Which object was opened by the person?,AZFKK,0.7,8.8,The box.,"[{'choice_id': 0, 'choice': 'The bag.', 'choic...",In frame 0: person_00ac wearing pant_31fc.pers...,The laptop.,Wrong,NaN
10,216,453,Interaction_T1_3791,Which object was thrown by the person?,AZFKK,0.0,3.4,The box.,"[{'choice_id': 0, 'choice': 'The shoe.', 'choi...",In frame 0: person_de87 has arm_16e4.person_de...,"Based on the given frames, none of the objects...",Wrong,NaN
13,224,470,Interaction_T1_3954,Which object was closed by the person?,7SXQS,1.0,7.0,The window.,"[{'choice_id': 0, 'choice': 'The laptop.', 'ch...",In frame 0: man_5d67 in shirt_b8ae.man_5d67 we...,The door was closed by the person.,Wrong,NaN
18,169,361,Interaction_T1_3022,Which object was taken by the person?,1RNK6,7.2,12.5,The box.,"[{'choice_id': 0, 'choice': 'The phone/camera....",In frame 1: handle_4896 on door_c7e6.door_c7e6...,"Based on the given frames, there is no informa...",Wrong,NaN
21,159,339,Interaction_T1_2852,Which object was put down by the person?,FM078,12.3,19.4,The box.,"[{'choice_id': 0, 'choice': 'The sandwich.', '...",In frame 0: man_d28d wearing shirt_b139.man_d2...,"Based on the given information, the person did...",Wrong,NaN


In [24]:
sequence_ds = ds.loc[ds['question_id'].str.contains('Sequence')]
sequence_ds.name = 'Sequence'
sequence_ds.head()

,Unnamed: 0.1,Unnamed: 0,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,Match,new_Match
47,1015,2476,Sequence_T1_737,Which object did the person put down after the...,6ZWSU,0.0,7.2,The book.,"[{'choice_id': 0, 'choice': 'The bag.', 'choic...",In frame 0: bottle_43cf on table_cfc3.woman_9e...,"Based on the description provided, it is not m...",Wrong,NaN
48,1023,2490,Sequence_T1_962,Which object did the person throw after they c...,136V6,1.1,10.5,The shoe.,"[{'choice_id': 0, 'choice': 'The towel.', 'cho...",In frame 0: curtain_f36b covering window_5da6....,"Based on the given frames, there is no informa...",Wrong,NaN
56,1505,2448,Sequence_T1_442,Which object did the person put down after the...,C1DK7,4.2,21.6,The phone/camera.,"[{'choice_id': 0, 'choice': 'The bag.', 'choic...",In frame 0: laptop_11a4 on table_6fdf.laptop_1...,"According to the given information, there is n...",Wrong,NaN
57,1506,2449,Sequence_T1_443,Which object did the person close after they h...,C1DK7,0.0,10.2,The laptop.,"[{'choice_id': 0, 'choice': 'The closet/cabine...",In frame 0: man_24f1 sitting on chair_5165.man...,"Based on the description, the person did not c...",Wrong,NaN
58,1507,2450,Sequence_T1_444,Which object did the person close after they s...,C1DK7,0.0,10.2,The laptop.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",In frame 0: man_4de2 sitting on chair_12cd.man...,"Based on the description in the frames, there ...",Wrong,NaN


In [25]:
prediction_ds = ds.loc[ds['question_id'].str.contains('Prediction')]
prediction_ds.name='Prediction'
prediction_ds.head()

,Unnamed: 0.1,Unnamed: 0,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,Match,new_Match
0,4251,5989,Prediction_T1_88,What will the person do next?,DGPAW,0.0,12.025,Open the door.,"[{'choice_id': 0, 'choice': 'Put down the cup/...",In frame 0: man_9076 wearing jean_b44d.man_907...,"Based on the description provided, there is no...",Wrong,Correct
1,4252,5991,Prediction_T1_110,What will the person do next?,D1WYU,7.2,21.525,Put down the shoe.,"[{'choice_id': 0, 'choice': 'Open the closet/c...",In frame 0: person_c513 has arm_d428.person_c5...,"Based on the information provided, there is no...",Wrong,Wrong
2,4253,5992,Prediction_T1_111,What will the person do next?,D1WYU,0.0,9.000,Throw the bag.,"[{'choice_id': 0, 'choice': 'Open the laptop.'...",In frame 0: bottle_602a on table_0093.man_abe5...,"From the given information, there is no mentio...",Wrong,Correct
6,4559,6530,Prediction_T4_1185,Which object would the person close next after...,6UDGS,5.5,17.925,The door.,"[{'choice_id': 0, 'choice': 'The closet/cabine...",In frame 0: man_2a5b wearing shirt_25a6.man_2a...,The option would be: The door.,Correct,Correct
49,4524,6486,Prediction_T4_675,Which object would the person eat next after t...,432NL,4.5,26.825,The medicine.,"[{'choice_id': 0, 'choice': 'The door.', 'choi...",In frame 0: man_760e wearing shirt_e98c.man_76...,"Based on the given information, the next objec...",Wrong,NaN


In [26]:
feasibility_ds = ds.loc[ds['question_id'].str.contains('Feasibility')]
feasibility_ds.name= 'Feasibility'
feasibility_ds.head()

,Unnamed: 0.1,Unnamed: 0,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,Match,new_Match
28,4775,6836,Feasibility_T5_1329,Which object is the person able to put down af...,GBMNT,0.0,5.6,The bag.,"[{'choice_id': 0, 'choice': 'The dish.', 'choi...",In frame 0: man_3d9f wearing shirt_d31e.man_3d...,"Based on the information provided, we cannot d...",Wrong,NaN
41,4903,7007,Feasibility_T6_621,What is the person able to do after sitting on...,JZDL6,9.0,14.0,Hold the shoe.,"[{'choice_id': 0, 'choice': 'Hold the shoe.', ...",In frame 0: cup_1cb9 on table_4634.book_df16 o...,"Based on the given frames, there is no mention...",Wrong,NaN
42,4914,7020,Feasibility_T6_754,What is the person able to do after walking th...,GBMNT,0.0,5.6,Put down the bag.,"[{'choice_id': 0, 'choice': 'Put down the cup/...",In frame 0: man_f30e wearing shirt_30d7.man_f3...,"Based on the given information, there is no in...",Wrong,NaN
43,4918,7026,Feasibility_T6_937,What is the person able to do after putting do...,9K2TY,19.2,23.9,Throw the towel.,"[{'choice_id': 0, 'choice': 'Tidy up the blank...",In frame 0: man_0805 wearing short_5f53.man_08...,"Based on the given information, it is not spec...",Wrong,NaN
114,4619,6610,Feasibility_T2_22,What else is the person able to do with the bag?,GKH0F,7.0,13.0,Put down the bag.,"[{'choice_id': 0, 'choice': 'Hold the bag.', '...",In frame 0: man_85f3 wearing shirt_c38c.man_85...,"The person is able to hold the bag, put down t...",Correct,NaN


In [27]:
for i in [ds, interaction_ds,sequence_ds,prediction_ds, feasibility_ds]:
  count = (i['new_Match'].value_counts()['Correct'])
  length = len(i)
  print('length of {} is: {}'.format(i.name, length))
  print('Correct match percentage for {} is: {}'.format(i.name, count/length))

length of all data is: 124
Correct match percentage for all data is: 0.024193548387096774


KeyError: ignored

In [35]:
context = 'In frame 0: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 has hand_3b94.woman_6109 has hand_f64a.shirt_3a73 on woman_6109.hair_cfe9 on woman_6109.pant_b4d2 on woman_6109.head_366f of woman_6109.arm_2c4a of woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.hand_f64a of woman_6109.In frame 1: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 has hand_f64a.woman_6109 has arm_2c4a.woman_6109 has hand_7c4d.woman_6109 has hand_a7ca.shirt_3a73 on woman_6109.hair_cfe9 on woman_6109.head_366f of woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 over window_3e58.bottle_13db on counter_6d66.In frame 2: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 has hand_4822.shirt_3a73 on woman_6109.hair_cfe9 on woman_6109.bottle_c5b4 on counter_6d66.bottle_13db on counter_6d66.In frame 3: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 has arm_2c4a.woman_6109 has hand_4822.woman_6109 has hand_2613.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 over window_3e58.In frame 4: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 wearing glass_e33c.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 over window_3e58.bottle_13db on counter_6d66.glass_e33c on woman_6109.In frame 5: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 wearing glass_e33c.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 over window_3e58.bottle_13db on counter_6d66.glass_e33c on woman_6109.In frame 6: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 on window_3e58.bottle_13db on counter_6d66.bottle_13db on counter_314c.In frame 7: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 on window_3e58.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.In frame 8: bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.sink_af57 on counter_314c.curtain_35a8 over window_3e58.bottle_13db on counter_6d66.bottle_13db on counter_314c.bottle_13db near sink_af57.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.In frame 9: bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.girl_eb23 has hand_01f2.In frame 10: bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.bottle_13db on counter_6d66.bottle_13db on counter_314c.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.girl_eb23 has hand_8cde.In frame 11: shirt_3a73 on girl_eb23.bottle_c5b4 on counter_6d66.sink_af57 on counter_6d66.curtain_35a8 on window_d1db.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.girl_eb23 sitting on chair_29c0.chair_29c0 at table_c710.chair_29c0 at counter_4559.In frame 12: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 on chair_1702.woman_6109 has hand_4822.woman_6109 has hat_44a8.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.bottle_13db on counter_6d66.bottle_13db on counter_314c.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.chair_29c0 at table_c710.In frame 13: shirt_3a73 on girl_eb23.hair_cfe9 on girl_eb23.pant_b4d2 on girl_eb23.bottle_c5b4 on counter_314c.curtain_35a8 on window_d1db.bottle_13db on counter_6d66.bottle_13db on counter_314c.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.girl_eb23 has hair_4439.In frame 14: shirt_3a73 on girl_eb23.hair_cfe9 on girl_eb23.pant_b4d2 on girl_eb23.bottle_c5b4 on counter_6d66.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.girl_eb23 has hair_4439.man_f230 wearing hat_7a75.man_f230 wearing shirt_5427.In frame 15: shirt_3a73 on girl_eb23.hair_cfe9 on girl_eb23.pant_b4d2 on girl_eb23.curtain_35a8 on window_d1db.bottle_13db on counter_6d66.girl_eb23 in shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_1702.In frame 16: shirt_3a73 on girl_eb23.pant_b4d2 on girl_eb23.bottle_c5b4 on counter_6d66.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.man_f230 wearing shirt_5427.In frame 17: shirt_3a73 on girl_eb23.pant_b4d2 on girl_eb23.bottle_c5b4 on counter_6d66.curtain_35a8 on window_d1db.bottle_13db on counter_6d66.bottle_13db on counter_314c.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_1702.girl_eb23 holding bottle_13db.chair_29c0 at counter_48c0.In frame 18: shirt_3a73 on girl_eb23.pant_b4d2 on girl_eb23.bottle_13db on counter_6d66.bottle_13db on counter_86ea.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_1702.girl_eb23 sitting on chair_29c0.In frame 19: woman_6109 wearing pant_b4d2.shirt_3a73 on girl_eb23.pant_b4d2 on girl_eb23.bottle_c5b4 on counter_6d66.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_1702.girl_eb23 sitting on chair_29c0.In frame 20: woman_6109 wearing shirt_3a73.woman_6109 wearing pant_b4d2.shirt_3a73 on girl_eb23.pant_b4d2 on girl_eb23.bottle_c5b4 on counter_6d66.curtain_35a8 on window_d1db.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 on chair_1702.girl_eb23 has hand_8cde.girl_eb23 sitting on chair_29c0.In frame 21: woman_6109 has hair_cfe9.woman_6109 sitting on chair_29c0.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_29c0.girl_eb23 sitting on chair_09d0.In frame 22: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 sitting on chair_1702.woman_6109 sitting on chair_29c0.woman_6109 sitting on chair_ccea.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.curtain_35a8 on window_d1db.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_29c0.chair_29c0 at table_c710.chair_ccea at table_c710.In frame 23: woman_6109 in shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 sitting on chair_ccea.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_13db on counter_6d66.chair_ccea at table_c710.In frame 24: woman_6109 in shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 sitting on chair_1702.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_13db on counter_6d66.chair_ccea at table_c710.In frame 25: woman_6109 has hair_cfe9.woman_6109 sitting on chair_1702.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.chair_ccea at table_c710.boy_8d1a wearing shirt_3a73.boy_8d1a wearing pant_b4d2.boy_8d1a has hair_cfe9.In frame 26: woman_6109 has hair_cfe9.bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 has hand_01f2.girl_eb23 sitting on chair_ccea.chair_ccea at table_c710.boy_8d1a wearing shirt_3a73.boy_8d1a wearing pant_b4d2.boy_8d1a has hair_cfe9.In frame 27: bottle_13db on counter_6d66.girl_eb23 wearing shirt_3a73.girl_eb23 wearing pant_b4d2.girl_eb23 has hair_cfe9.girl_eb23 sitting on chair_ccea.chair_ccea at table_c710.In frame 28: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.shirt_3a73 on woman_6109.pant_b4d2 on woman_6109.bottle_c5b4 on counter_6d66.bottle_13db on counter_6d66.chair_ccea at table_c710.boy_8d1a wearing shirt_3a73.boy_8d1a wearing pant_b4d2.boy_8d1a has hair_cfe9.glass_5a0b on table_c710.In frame 29: woman_6109 wearing shirt_3a73.woman_6109 has hair_cfe9.woman_6109 wearing pant_b4d2.woman_6109 sitting on chair_1702.bottle_c5b4 on counter_314c.curtain_35a8 on window_d1db.bottle_13db on counter_314c.chair_ccea at table_c710.boy_8d1a wearing shirt_3a73.boy_8d1a wearing pant_b4d2.boy_8d1a has hair_cfe9.plant_5bab on counter_314c.curtain_f5df on window_3e58.'
answer = run_gpt(context, 'Which object did the person put down after they watched outside the window? Guess the most likely answer among these four options:  (1) shoe. (2) cup/glass/bottle. (3) towel. (4) box. Respond only with a single number between 1 and 4. Do not produce any other output.')
print('OIC answer: {}'.format(answer))

OIC answer: 2
